# Finding CRISPR arrays
## Specification
## Program Name: randomizedMotifSearch.py

## Input/Output: STDIN, STDOUT

## Options:

- -i iterations (int)
- -k motif length (int)
- -p pseudocount (float)

In [ ]:
Ex: python randomizedMotifSearch.py -i 1000 -k 13 -p 1 < input.fa > output.out

Inspection Notebook example:
main ( "input.fa", options = [ "-i 1000", "-k 13", "-p 1"] )

## Background
CRISPR arrays exist in most archaeal species and many bacterial species. The arrays are encoded in chromosomal DNA as direct repeats (DR) that flank sequences (spacers) that provide immunity against invading plasmids, viral sequences, and other foreign DNA .. and RNA. These arrays can be short, comprising 1-2 spacers, or they can contain hundreds of spacer elements. The arrays are punctuated by these DR sequences. The promoter element seems to be upstream of the array, and initiates transcription of the associated RNA strand that is then cleaved and those spacers are loaded into protein machines that provide specific targeting of the associated invader should it be found. That targeting causes cleavage of the foreign DNA in known cases.

There are three main classes of CRISPR systems; these are type I,II and III, and are established based on the associated protein machines - actually the specific genes that encode for them. In the archaea, only type I and III have been found so far, while in bacteria, all three types have been seen. One of those types, type II, exists in a few species, including _Streptococcus pyogenes_, which encodes for a type II system. The specific gene that is used to classify this system is named Cas9.

We see cases where Cas genes, and sometimes CRISPR arrays, exist in a viral sequence. This, of course, opens many questions about the role of the CRISPR system with respect to viruses - why would a virus carry an immune defense that targets viral sequence? The existence of these arrays suggests that the CRISPR system itself might be mobile. How does the system get established, and how does the surrounding transcription machinery get encoded on the chromosome?

For this assignment, we will work to find the promoter motif. We would expect that this promoter motif and an associated B recognition element (BRE) would be present in archaeal species. What is that motif, and where is it located relative to transcription start?

We know little about the sequence specifics of the promoter element, though it seems to be about 13 nucleotides in length. In Pyrobaculum species, genes are packed quite tightly, so we can look upstream of the array by about 50 bases to see if a common motif sequence might exist. We need to assume that some mutations can be tolerated in that sequence.

## Null model
In prior years, we included an option that scrambled the input sequences before running to find a best score. This would effectively eliminate any signal that might be present in the data, while preserving the overall composition of the sequences. We would then compare the best score achieved without scrambling to the score achieved with scrambling. The information that remains in a scrambled set of sequences is only the base level composition of the sequences:

$Q=Pr_{s\in\lbrace{ACTG}\rbrace}(s)=\frac{count(s)}{N}$

We can then use relative entropy to find a score relative to the base level composition by:

$REscore = \sum_{i=0}^{cols}\sum^{j \in ACGT}P_{i}(j)\log_{2}(\frac{P_{i}(j)}{Q(j)} )$

## Assignment
__Write a BME205 style program (.py file)__ that is based on the Randomized Motif Search that presented in class/videos and is described in Chapter 2 of the text. Your program should accept a fasta file that contains multiple fasta records as input (STDIN), and your program will then output (STDOUT) the consensus sequence and associated profile score. The score will be the sum of encoding costs (entropies) across each position in the final profile. We will use pseudocounts for this assignment, as described in the text ( p. 86-89), and we will need an option to provide the number of pseudocounts (-p).

__Produce a notebook that should be fully runable for use during inspections.__

I find it easier to accumulate counts rather than calculating a probability based profile. This will also make it a bit easier to deal with pseudocounts when scoring your count-based profile.

$score = \sum_{i=0}^{cols}\sum^{j \in ACGT}P_{i}(j)\log_{2}(\frac{P_{i}(j)}{Q(j)} )$

Randomized Motif Search (p. 93) can be easily trapped in local minima, so we will need to iterate some number of times to find a trajectory that produces the best results. This will need to be an option (-i) that establishes the iteration number.

We also don't know the appropriate motif length that we are looking for, so we need an option (-k) to specify this.

The full command line string should then look something like this:

In [ ]:
randomizedMotifSearch.py -i=100000 -p=1 -k=13 <somefile.fa >someOutputFile.fa

your program will need to use standard OO style, include docstrings and be well commented. It must run! 

I will provide a few fasta files that present the upstream 50 bases from species of the Pyrobaculum group. It is possible that the promotor+BRE that we are looking for is conserved across these species, maybe not perfectly though. Providing more data to a single run may be useful.

I do have expression data for these species, so we know that not all of these arrays are functional - at least under the conditions when I grew them.

Your program should output the final score achieved, along with the associated consensus motif.

## Extra Credit
The following optional features would be useful and considered for a combined extra 5 points:

1) -g Use Gibbs sampling to find the optimal consensus motif.

2) -m Print the specific motif and the name of the contributing sequence for each of the participating fasta records.

Each of these options should only be true when the flag is specified by the user (ie. you shouldn't have to explicitly state True/False on the commandline).

Submit your working program to Canvas.

## Notes:

1) It is likely that we will need to amend this assignment as we work with it.

2) Sept-2021 eliminated the need for scrambling sequences by using relative entropy

## Inspection intro

- What data structure did you use to hold your sequence motifs (DNA)? 
- Is your profile  structure based on counts or frequencies? 
- Where did you implement iterations (mostly why did you choose this)?
- If you implemented Gibbs Sampling, how did you implement recomputing of the profile and motif?

## Inspection Results

In [3]:
# Randomized Motif Search
# input: A fasta file with multiple fasta records
# returns the consensus sequence and associated profile score
# Groupmates: Alan Faber, Sarah Ghasemi
# Author: Jeffrey Jacob

import sys
from random import random
import random
import math
class FastAreader:
    ''' This class reads in the FastA file and provides the main the header and sequence in the FastA file'''
    def __init__(self, fname=''):
        '''contructor: saves attribute fname '''
        self.fname = fname
    def doOpen(self):
        ''' Opens the file in stdin '''
        if self.fname is '':
            return sys.stdin
        else:
            return open(self.fname)
    def readFasta(self):
        ''' Reads the file and returns the header and sequence one by one '''
        header = ''
        sequence = ''
        with self.doOpen() as fileH:
            header = ''
            sequence = ''
            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>'):
                line = fileH.readline()
            header = line[1:].rstrip()
            for line in fileH:
                if line.startswith('>'):
                    yield header, sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else:
                    sequence += ''.join(line.rstrip().split()).upper()
        yield header, sequence

class CommandLine():
    ''' Deals with the commandLine options provided and sets the values to be used by main and other classes '''
    def __init__(self, inOpts=None):
        ''' Contains all the information that either saves the information given by the user, or uses the default values'''
        import argparse
        self.parser = argparse.ArgumentParser(description='This program find the motif with the highest relative entropy score', add_help=True)
        self.parser.add_argument('-i', '--iterations', type=int, action='store', default=1000, dest='iterations',
                                 help='Provide the number of runs the motif search should take, the default is 1000 iterations.') #handles the iterations option provided by the user, or set as the default
        self.parser.add_argument('-p', '--pseudoCount', type=float, action='store', default=1,dest='pseudoCount',
                                 help='Provide the psuedo count, the default is 1 pseudo count.') #handles the pseudo count option provided by the user, or set as the default
        self.parser.add_argument('-k', '--kmerLength', type=int, action='store', default=13, dest='kmerLength',
                                 help='Provide the kmer length') #handles the kmer length option provided by the user, or set as the default
        if inOpts is None:
            self.args = self.parser.parse_args()
        else :
            self.args = self.parser.parse_args(inOpts)

class Calculations:
    ''' The calculations class deals with all the processing related to the sequences '''
    def __init__(self):
        ''' Description: Initializes all the values that will be utilized within the class '''
        ''' Input: The method does not have any inputs '''
        ''' Output: The method initializes certain values that will be changed in future methods '''
        self.aCount = 0; self.tCount = 0; self.cCount = 0; self.gCount = 0 # Initializes the counts of all the nucleotides
        self.aNull = 0; self.tNull = 0; self.cNull = 0; self.gNull = 0 # Initializes the null values of all the nucleotides
        self.total = 0 
        self.profArray = [] 

    def createNull(self, a, t, c, g, pCount):
        ''' Description: Calculates the Null Model '''
        ''' Input: This method uses the number of A, T, C and G and the pseudo count '''
        ''' Output: The method saves the null values of each of the nucleotides to be used in other methods '''
        self.aCount += a; self.tCount += t; self.cCount += c; self.gCount += g 
        self.total = a + t + c + g + (pCount*4) 
        self.aNull = ((a+pCount)/self.total); self.tNull = ((t+pCount)/self.total); self.cNull = ((c+pCount)/self.total); self.gNull = ((g+pCount)/self.total) # Calculates the null probabilities of each of the nucleotides

    def randomMotif(self, seqList, kmerLength):
        ''' Description: Retrieves random motifs of a certain length in each of the sequences '''
        ''' Input: This method uses a list of sequences and the length of the k-mer which is specified by the user '''
        ''' Output: The method returns a random motif within each of the sequences '''
        totalLength = len(seqList[0])
        ranList = [random.randint(0, totalLength-(kmerLength+1)) for i in seqList] 
        motifList = []; count = 0
        for i in seqList:
            motifList.append(i[ranList[count]:ranList[count]+kmerLength]) # Finds and saves the selected motif into a list of motifs
            count+=1
        return motifList

    def createProfile(self, motifList, kmerLength, pseudoCount):
        ''' Description: This method determines the profile given the list of motifs and length of the k-mer '''
        ''' Input: The input for this method is the list of motif, the length of the k-mer, and the pseudo count '''
        ''' Output: The method returns the profile '''
        self.profArray = []; subList = []
        for i in range(kmerLength): # Creates an empty list of lists
            subList.append([])
        numMotif = len(motifList)
        aProfList = []; tProfList = []; cProfList = []; gProfList = [];
        for i in range(kmerLength): # Creates a 2D array to create the profile by counting the number of nucleotides in specific positions
            for j in range(numMotif):
                subList[i].append(motifList[j][i]) 
            aProfList.append((subList[i].count("A")+pseudoCount)/(numMotif+(pseudoCount*4)))
            tProfList.append((subList[i].count("T")+pseudoCount)/(numMotif+(pseudoCount*4)))
            cProfList.append((subList[i].count("C")+pseudoCount)/(numMotif+(pseudoCount*4)))
            gProfList.append((subList[i].count("G")+pseudoCount)/(numMotif+(pseudoCount*4)))
        self.profArray.append(aProfList); self.profArray.append(tProfList); self.profArray.append(cProfList); self.profArray.append(gProfList) # Places the probabilities into a profile array
        return self.profArray

    def iterMotifs(self, seqList, kmerLength):
        ''' Description: Determines all the motifs in a given set of sequences, and determines a motif with the highest score given the profile '''
        ''' Input: The method uses the all the sequences and the length of the k-mer '''
        ''' Output: The output is the final motif that has the highest value '''
        totalLength = len(seqList[0])
        finalMotif = []
        for i in range(len(seqList)): # Iterates through all the sequences to find the best possible motif for each sequence
            motifList = []
            for j in range(totalLength-kmerLength): # Finds all the possible motifs in a given sequence
                motifList.append(seqList[i][j:j+kmerLength])
            motifDict = {}
            for k in range(len(motifList)): # Iterates through all the motifs and calculates based on the profile
                count = 1
                for l in range(kmerLength): # Iterates through the length of the kmer and calculates it's score based on profile
                    if (motifList[k][l]) == "A": count *=(self.profArray[0][l])
                    elif (motifList[k][l]) == "T": count *= (self.profArray[1][l])
                    elif (motifList[k][l]) == "C": count *= (self.profArray[2][l])
                    elif (motifList[k][l]) == "G": count *= (self.profArray[3][l])
                motifDict[motifList[k]] = count # Places the motif and it's correpsonding score in a dictionary
            finalMotif.append(max(motifDict, key=motifDict.get)) # Determines the motif with the highest value and adds it to a list of motifs
        return finalMotif

    def relativeEntropy(self, profile, consensus):
        ''' Description: Calculates the relative entropy of the consensus sequence '''
        ''' Input: The function uses the profile of the final motif and the determined consensus '''
        ''' Output: The method returns the relative entropy score of the consensus '''
        score = 0
        for i in range(len(consensus)): # Goes through all the nucleotides within the consensus and calculates the total relative entropy score
            if consensus[i] == "A":
                score += (profile[0][i]) * (math.log2((profile[0][i])/self.aNull))
            elif consensus[i] == "T":
                score += (profile[1][i]) * (math.log2((profile[1][i])/self.tNull))
            elif consensus[i] == "C":
                score += (profile[2][i]) * (math.log2((profile[2][i])/self.cNull))
            elif consensus[i] == "G":
                score += (profile[3][i]) * (math.log2((profile[3][i])/self.gNull))
        return (score)

def main (inFile, myCommandLine=None):
    ''' Description: Runs the entire program by reading the command line and outputting the corresponding profile and relative entropy score '''
    ''' Input: The input is the options given by the user through the commandLine '''
    ''' Output: The main function prints out the consensus with the highest relative entropy score and its score into the specified output file '''
    if myCommandLine is None:
        myCommandLine = CommandLine()  # read options from the command line
    else:
        myCommandLine = CommandLine(
            myCommandLine)  # interpret the list passed from the caller of main as the commandline.
    
    seqList = []
    aCount = 0; tCount = 0; cCount = 0; gCount = 0
    for head, seq in FastAreader(inFile).readFasta(): #iterates through all the sequences within the Fasta
        processedSeq = Calculations()
        seqList.append(seq)
        aCount += seq.count("A"); tCount += seq.count("T"); cCount += seq.count("C"); gCount += seq.count("G")
    processedSeq.createNull(aCount, tCount, cCount, gCount, myCommandLine.args.pseudoCount) # Determines the Null Model given the counts of nucleotides within the sequences
    allEntropy = []; allConsensus = []
    for i in range(myCommandLine.args.iterations): # Iterates through the number of user provided user iterations and outputs the relative entropy score along with its corresponding consensus
        oldMotifList = processedSeq.randomMotif(seqList, myCommandLine.args.kmerLength) # Creates the initial motif list using the random motif generator
        processedSeq.createProfile(oldMotifList, myCommandLine.args.kmerLength, myCommandLine.args.pseudoCount)
        newMotifList = processedSeq.iterMotifs(seqList, myCommandLine.args.kmerLength)
        while oldMotifList != newMotifList: # Determines the best motif list which is attained with the new and old motif list are the same
            oldMotifList = newMotifList 
            profile = processedSeq.createProfile(oldMotifList, myCommandLine.args.kmerLength, myCommandLine.args.pseudoCount) # Finds the profile of the last motif list
            newMotifList = processedSeq.iterMotifs(seqList, myCommandLine.args.kmerLength) # Saves the motif list to a newMotifList to compare
        consensus = []
        for i in range(myCommandLine.args.kmerLength): # Iterates through the length of the kmer and determines the consensus sequence
            aVal = profile[0][i]; tVal = profile[1][i]; cVal = profile[2][i]; gVal = profile[3][i] 
            maxNuc = ([aVal, tVal, cVal, gVal]).index(max([aVal, tVal, cVal, gVal])) # Determines which nucleotide was in the highest frequence
            if maxNuc == 0:nucVal = "A"
            elif maxNuc == 1:nucVal = "T"
            elif maxNuc == 2:nucVal = "C"
            elif maxNuc == 3:nucVal = "G"
            consensus.append(nucVal) # Appends the nucleotide to the consensus
        entropyScore = processedSeq.relativeEntropy(profile, consensus) 
        allConsensus.append(consensus) # Appends the consenses to a list of consensus's 
        allEntropy.append(entropyScore) 
    finalEntropy = max(allEntropy) # Finds the highest entrpy score
    finalConsensus = "".join(allConsensus[allEntropy.index(finalEntropy)]) 
    print(str(finalConsensus) + "   " + str(finalEntropy)) # Prints the consensus and it's entropy score to the output file

if __name__== "__main__":
    main("pogCrisprs", ["-i 1000", "-p 3", "-k 8"])
    

<>:25: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:25: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-2bae87261092>:25: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.fname is '':


AGAACTGT   2.984359622300305
